<p style="text-align: center">
    <img src="../../assets/images/untref-logo-negro.svg" style="height: 50px;" />
</p>

<h3 style="text-align: center">Estructuras de Datos</h3>

<h2 style="text-align: center">Clase 6: Camino mínimo en grafos dirigidos</h3>

## Camino Mínimo

Dado un grafo dirigido encontrar el camino mínimo desde un vértice dado a todos los demás vértices del grafo.

### Métodos

- Grafo dirigido sin pesos: Se resuelve con un recorrido **BFS**.
- Grafo dirigido con pesos no negativos: **Algoritmo de Dijkstra**.
- Grafo dirigido con pesos negativos, pero sin ciclos negativos: **Algoritmo de Bellman-Ford**.

### Propiedades de los caminos mínimos

- Los pesos pueden ser distancia, costo, tiempo, o cualquier otro parámetro que necesitemos modelar.
- No es necesario que todos los nodos sean alcanzables.
- Las aristas con pesos negativos, traen problemas.
- Los caminos mínimos normalmente son "caminos simples".
- Los caminos mínimos no son necesariamente únicos.

### Algoritmo de Dijkstra

Encontrar el camino mínimo desde $A$ hacia todos los demás vértices del grafo:

> El grafo no puede tener aristas con pesos negativos.

<p style="text-align: center">
    <img src="figuras/grafo-dijkstra.png" style="width: 500px;" />
</p>

#### Algoritmo

|Referencia|Descripción|
|--:|---|
|`s`| Nodo inicial|
|`distancia[v]`|Distancia desde `s` hasta el nodo `v`|
|`previo[v]`|Empezando por `s`, guarda cual es el nodo anterior a `v` en el camino `s-v`|
|`visitado[v]`|Verdadero si `v` ya fué visitado|

<table style="width: 100%;">
    <tbody>
        <tr>
            <td style="width: 60%;">

```
DIJKSTRA (G: DiGrafo, s: Vertice)
    pq <- Cola de prioridad de mínimo vacía

    PARA CADA v EN G.nodos
        distancia[v] = ∞
        previo[v] = None
        visitado[v] = False

    distancia[s] = 0
    pq.encolar(s, 0)

    MIENTRAS NO pq.est_vacia:
        (v, distancia[v]) = pq.desencolar()

        visitado[v] = True

        PARA CADA w EN v.nodos_adyacente:
            SI w no está visitado:
                SI distancia[v] + peso(v, w) < distancia[w]:
                    distancia[w] = distancia[v] + peso(v, w)
                    previo[w] = v
                    pq.encolar(w, distancia[w])
```
</td>
            <td>
                <p style="text-align:center;">
                    <img src="figuras/grafo-dijkstra.png" style="width:500px;" />
                </p>
            </td>
        </tr>
    </tbody>
</table>


#### Ejemplo

In [ ]:
from math import inf

from edd.coladeprioridad import ColaDePrioridad
from edd.grafo import DiGrafo, Vertice


def dijkstra_por_pasos(self, s: Vertice):
    pq = ColaDePrioridad()

    distancia = {v: inf for v in self.vertices}
    previo = {v: None for v in self.vertices}
    visitado = {v: False for v in self.vertices}
    aristas = []

    distancia[s] = 0
    pq.encolar(s, distancia[s])

    yield {
        "msj": f"Encolamos {s.id} como nodo inicial",
        "cola": pq,
        "distancia": distancia,
        "previo": previo,
        "visitado": visitado,
        "aristas": aristas,
    }  ### IGNORAR

    while not pq.esta_vacia():
        v, _ = pq.desencolar()

        visitado[v] = True

        yield {
            "msj": f"Desencolamos y visitamos {v.id}",
            "cola": pq,
            "distancia": distancia,
            "previo": previo,
            "visitado": visitado,
            "aristas": aristas,
        }  ### IGNORAR

        for a in v.aristas:
            aristas.append((a.origen.id, a.destino.id))
            w = a.destino

            if not visitado[w] and distancia[v] + a.peso < distancia[w]:
                distancia[w] = distancia[v] + a.peso
                previo[w] = v
                pq.encolar(w, distancia[w])
                msj = f"Encolamos {w.id} con la nueva distancia {distancia[w]}"
            else:
                msj = f"No se mejoró la distancia a {w.id}"

            yield {
                "msj": msj,
                "cola": pq,
                "distancia": distancia,
                "previo": previo,
                "visitado": visitado,
                "aristas": aristas,
            }  ### IGNORAR


# Hacemos "monkey patching" del método que acabamos de implementar.
DiGrafo.dijkstra_por_pasos = dijkstra_por_pasos

In [ ]:
# Ejecutar esto una vez para inicializar el grafo.
from edd.grafo import DiGrafo

G = DiGrafo()
G.agregar_arista("A", "B", 4)
G.agregar_arista("A", "C", 1)
G.agregar_arista("B", "E", 3)
G.agregar_arista("C", "B", 2)
G.agregar_arista("C", "D", 2)
G.agregar_arista("D", "E", 3)

pasos = G.dijkstra_por_pasos(G["A"])

In [ ]:
# Cada vez que se ejecute esta celda, se mostrará una iteración del algoritmo de Dijkstra.
from edd.jp import build_html_table, render_html

try:
    estado = next(pasos)
except StopIteration:
    print("~ Fin ~\n\n")

    headers = ["Vertice", "Distancia", "Previo", "Visitado"]
    rows = [
        [
            v.id,
            estado["distancia"][v],
            estado["previo"][v].id if estado["previo"][v] else "-",
            "Si" if estado["visitado"][v] else "No",
        ]
        for v in G.vertices
    ]
    render_html(build_html_table(headers, rows))
finally:
    print(f">>> {estado['msj']}\n")
    print(f"pq = {estado['cola']}\n")

    node_labels = {
        v.id: f"({estado['distancia'][v]}, {estado['previo'][v].id if estado['previo'][v] else None})"
        for v in G.vertices
        # if estado["visitado"][v]
    }
    G.draw(
        highlight_edges=estado["aristas"],
        highlight_nodes=[v.id for v, visitado in estado["visitado"].items() if visitado],
        node_labels=node_labels,
    )

#### Complejidad del algoritmo

```
DIJKSTRA (G: DiGrafo, s: Vertice)
    pq <- Cola de prioridad de mínimo vacía

    PARA CADA v EN G.nodos  ### O(|V|)
        distancia[v] = ∞
        previo[v] = None
        visitado[v] = False

    distancia[s] = 0
    pq.encolar(s, 0)

    MIENTRAS NO pq.est_vacia:  ### O(|V|+|A|)
        (v, distancia[v]) = pq.desencolar()  ### O(log(|v|))
        visitado[v] = True ### paso Greedy

        PARA CADA w EN v.nodos_adyacente:
            SI w no está visitado:
                SI distancia[v] + peso(v, w) < distancia[w]:
                    distancia[w] = distancia[v] + peso(v, w)
                    previo[w] = v
                    pq.encolar(w, distancia[w])
```

$$
\begin{align}
\mathcal{O}\left(\left|V\right|\right) + \mathcal{O}\left(\left|V\right| + \left|A\right|\right) \mathcal{O}\left(\log{\left|V\right|}\right) \\
\cancel{\mathcal{O}\left(\left|V\right|\right)} + \mathcal{O}\left(\left(\left|V\right| + \left|A\right|\right) \log{\left|V\right|}\right) \\
\mathcal{O}\left(\left(\cancel{\left|V\right|} + \left|A\right|\right) \log{\left|V\right|}\right) & \qquad ; \left|A\right| \leq \left|V\right|^2 \\
\boxed{\mathcal{O}\left(\left|A\right| \log{\left|V\right|}\right)}
\end{align}
$$

#### Grafos con aristas de pesos negativos

In [ ]:
# Ejecutar esto una vez para inicializar el grafo.
from edd.grafo import DiGrafo

G = DiGrafo()
G.agregar_arista("A", "B", -1)
G.agregar_arista("A", "C", 1)
G.agregar_arista("B", "E", 3)
G.agregar_arista("C", "B", 2)
G.agregar_arista("C", "D", 2)
G.agregar_arista("D", "E", 3)

G.draw()

pasos = G.dijkstra_por_pasos(G["A"])

In [ ]:
# Cada vez que se ejecute esta celda, se mostrará una iteración del algoritmo de Dijkstra.
from edd.jp import build_html_table, render_html

try:
    estado = next(pasos)
except StopIteration:
    print("~ Fin ~\n\n")

    headers = ["Vertice", "Distancia", "Previo", "Visitado"]
    rows = [
        [
            v.id,
            estado["distancia"][v],
            estado["previo"][v].id if estado["previo"][v] else "-",
            "Si" if estado["visitado"][v] else "No",
        ]
        for v in G.vertices
    ]
    render_html(build_html_table(headers, rows))
finally:
    print(f">>> {estado['msj']}\n")
    print(f"pq = {estado['cola']}\n")

    node_labels = {
        v.id: f"({estado['distancia'][v]}, {estado['previo'][v].id if estado['previo'][v] else None})"
        for v in G.vertices
        if estado["visitado"][v]
    }
    G.draw(
        highlight_edges=estado["aristas"],
        highlight_nodes=[v.id for v, visitado in estado["visitado"].items() if visitado],
        node_labels=node_labels,
    )

In [ ]:
# Ejecutar esto una vez para inicializar el grafo
from edd.grafo import DiGrafo

G = DiGrafo()
G.agregar_arista("A", "B", 1)
G.agregar_arista("A", "C", 1)
G.agregar_arista("B", "E", 3)
G.agregar_arista("C", "B", -2)
G.agregar_arista("C", "D", 2)
G.agregar_arista("D", "E", 3)

G.draw()

pasos = G.dijkstra_por_pasos(G["A"])

In [ ]:
# Cada vez que se ejecute esta celda, se mostrará una iteración del algoritmo de Dijkstra.
from edd.jp import build_html_table, render_html

try:
    estado = next(pasos)
except StopIteration:
    print("~ Fin ~\n\n")

    headers = ["Vertice", "Distancia", "Previo", "Visitado"]
    rows = [
        [
            v.id,
            estado["distancia"][v],
            estado["previo"][v].id if estado["previo"][v] else "-",
            "Si" if estado["visitado"][v] else "No",
        ]
        for v in G.vertices
    ]
    render_html(build_html_table(headers, rows))
finally:
    print(f">>> {estado['msj']}\n")
    print(f"pq = {estado['cola']}\n")

    node_labels = {
        v.id: f"({estado['distancia'][v]}, {estado['previo'][v].id if estado['previo'][v] else None})"
        for v in G.vertices
        if estado["visitado"][v]
    }
    G.draw(
        highlight_edges=estado["aristas"],
        highlight_nodes=[v.id for v, visitado in estado["visitado"].items() if visitado],
        node_labels=node_labels,
    )

### Algoritmo de Bellman-Ford

Encontrar el camino mínimo desde $A$ hacia todos los demás vértices del grafo:

> Ahora, el grafo puede tener aristas con pesos negativos (pero no ciclos negativos).

<p style="text-align: center">
    <img src="figuras/grafo-bellman-ford.png" style="width: 500px;" />
</p>

#### Algoritmo

<table style="width: 100%;">
    <tbody>
        <tr>
            <td style="width: 60%;">

```
BELLMAN_FORD (G: DiGrafo, s: Vertice)

    PARA CADA v EN G.nodos
        distancia[v] = ∞
        previo[v] = None

    distancia[s] = 0

    REPETIR len(G.nodos) - 1 VECES:
        PARA CADA (v, w, peso) EN G.aristas:
            SI distancia[v] + peso < distancia[w]
                distancia[w] = distancia[v] + peso
                previo[w] = v

    PARA CADA (v, w, peso) EN G.aristas:
        SI distancia[v] + peso < distancia[w]
            REPORTAR error: grafo con ciclos negativos
```
</td>
            <td>
                <p style="text-align:center;">
                    <img src="figuras/grafo-bellman-ford.png" style="width:500px;" />
                </p>
            </td>
        </tr>
    </tbody>
</table>


#### Ejemplo

In [ ]:
from math import inf

from edd.coladeprioridad import ColaDePrioridad
from edd.grafo import DiGrafo, Vertice


def bellman_ford_por_pasos(self, s: Vertice):
    distancia = {v: inf for v in self.vertices}
    previo = {v: None for v in self.vertices}
    iteraciones = {}

    distancia[s] = 0

    for i in range(1, len(self.vertices)):
        for a in self.aristas:
            if distancia[a.origen] + a.peso < distancia[a.destino]:
                distancia[a.destino] = distancia[a.origen] + a.peso
                previo[a.destino] = a.origen

        nodos = sorted(G.vertices, key=lambda v: v.id)
        iteraciones[i] = [(distancia[v], previo[v].id if previo[v] else None) for v in nodos]
        yield {
            "msj": f"En la iteración {i}.",
            "nodos": nodos,
            "aristas": [(a.origen.id, a.destino.id) for a in self.aristas],
            "iteraciones": iteraciones,
        }  ### IGNORAR


# Hacemos "monkey patching" del método que acabamos de implementar.
DiGrafo.bellman_ford_por_pasos = bellman_ford_por_pasos

In [ ]:
# Ejecutar esto una vez para inicializar el grafo.
from edd.grafo import DiGrafo

G = DiGrafo()
G.agregar_arista("A", "B", 1)
G.agregar_arista("A", "C", 1)
G.agregar_arista("B", "E", 3)
G.agregar_arista("C", "B", -2)
G.agregar_arista("C", "D", 2)
G.agregar_arista("D", "E", 3)

pasos = G.bellman_ford_por_pasos(G["A"])

In [ ]:
# Cada vez que se ejecute esta celda, se mostrará una iteración del algoritmo de Bellman-Ford.
from edd.jp import build_html_table, render_html

try:
    estado = next(pasos)
except StopIteration:
    print("~ Fin ~\n\n")
finally:
    print(f">>> {estado['msj']}\n")

    print("\n".join(str(a) for a in estado["aristas"]))

    headers = ["Iteración"] + [v.id for v in estado["nodos"]]
    rows = []
    for i, iteracion in estado["iteraciones"].items():
        rows.append([i] + [f"({d}, {p})" for d, p in iteracion])
        last_iteration = [f"({d}, {p})" for d, p in iteracion]

    node_labels = dict(zip((v.id for v in estado["nodos"]), last_iteration))
    G.draw(node_labels=node_labels)

    render_html(build_html_table(headers, rows))

#### Complejidad del algoritmo

```
BELLMAN_FORD (G: DiGrafo, s: Vertice)

    PARA CADA v EN G.nodos  ### O(|V|)
        distancia[v] = ∞
        previo[v] = None

    distancia[s] = 0

    REPETIR len(G.nodos) - 1 VECES: ### O(|V|)
        PARA CADA (v, w, peso) EN G.aristas:  ### O(|A|)
            SI distancia[v] + peso < distancia[w]
                distancia[w] = distancia[v] + peso
                previo[w] = v

    PARA CADA (v, w, peso) EN G.aristas:
        SI distancia[v] + peso < distancia[w]
            REPORTAR error: grafo con ciclos negativos
```

$$
\begin{align}
\mathcal{O}\left(\left|V\right|\right) + \mathcal{O}\left(\left|V\right|\right) \mathcal{O}\left(\left|A\right|\right) \\
\cancel{\mathcal{O}\left(\left|V\right|\right)} + \mathcal{O}\left(\left|V\right| \left|A\right|\right) \\
\boxed{\mathcal{O}\left(\left|V\right| \left|A\right|\right)}
\end{align}
$$